# Ensemble Learning

## Initial Imports

In [129]:
import warnings
warnings.filterwarnings('ignore')

In [130]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [131]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [132]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [133]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
label_encoder = LabelEncoder()

In [134]:
label_encoder.fit(df["home_ownership"])
df["home_ownership"] = label_encoder.transform(df["home_ownership"])

In [135]:
label_encoder.fit(df["verification_status"])
df["verification_status"] = label_encoder.transform(df["verification_status"])


In [136]:
label_encoder.fit(df["pymnt_plan"])
df["pymnt_plan"] = label_encoder.transform(df["pymnt_plan"])

In [137]:
label_encoder.fit(df["hardship_flag"])
df["hardship_flag"] = label_encoder.transform(df["hardship_flag"])

In [138]:
label_encoder.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = label_encoder.transform(df["debt_settlement_flag"])

In [139]:
label_encoder.fit(df["issue_d"])
df["issue_d"] = label_encoder.transform(df["issue_d"])
list(label_encoder.classes_)

['Feb-2019', 'Jan-2019', 'Mar-2019']

In [140]:

label_encoder.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = label_encoder.transform(df["next_pymnt_d"])
list(label_encoder.classes_)

['Apr-2019', 'May-2019']

In [141]:

label_encoder.fit(df["initial_list_status"])
df["initial_list_status"] = label_encoder.transform(df["initial_list_status"])

In [142]:

label_encoder.fit(df["application_type"])
df["application_type"] = label_encoder.transform(df["application_type"])

In [143]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


## Split the Data into Training and Testing

In [144]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [145]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [146]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [147]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [148]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [149]:
X_train = sc.fit_transform(X_train)

In [163]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = sc.fit(X_train)

In [165]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [182]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)
y_pred = brf.predict(X_test)

In [191]:
# Resample the training data with the BalancedRandomForestClassifier
# from sklearn.ensemble import RandomForestClassifier
#brf_model = RandomForestClassifier(n_estimators=100, random_state=1)
brf_model = brf.fit(X_train_scaled, y_train)

In [192]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
predictions = brf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9065387968613775


In [193]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,11,76
Actual 1,1532,15586


In [194]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.01      0.13      0.01        87
    low_risk       1.00      0.91      0.95     17118

    accuracy                           0.91     17205
   macro avg       0.50      0.52      0.48     17205
weighted avg       0.99      0.91      0.95     17205



In [202]:
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00        87
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17118

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



In [195]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09221642581945005, 'total_rec_prncp'),
 (0.06916241638945406, 'last_pymnt_amnt'),
 (0.0618766613650836, 'total_rec_int'),
 (0.05640233542880826, 'total_pymnt'),
 (0.05224191444301577, 'total_pymnt_inv'),
 (0.019668049914183097, 'dti'),
 (0.018487765914718957, 'installment'),
 (0.01757597588790243, 'out_prncp_inv'),
 (0.017474412796942576, 'mo_sin_old_rev_tl_op'),
 (0.01681202329811555, 'mo_sin_old_il_acct'),
 (0.016532635750652593, 'max_bal_bc'),
 (0.016226321311638158, 'out_prncp'),
 (0.015656115302157204, 'total_bal_ex_mort'),
 (0.015038910397363174, 'bc_util'),
 (0.014864887986300828, 'tot_cur_bal'),
 (0.014433875103802158, 'bc_open_to_buy'),
 (0.014238391796702715, 'mths_since_rcnt_il'),
 (0.0141228869481194, 'avg_cur_bal'),
 (0.013994765605835473, 'total_bal_il'),
 (0.01368162732395399, 'tot_hi_cred_lim'),
 (0.013571016666161603, 'total_rec_late_fee'),
 (0.013551923116788656, 'total_il_high_credit_limit'),
 (0.013492642633538552, 'total_bc_limit'),
 (0.013402981850705441, 'rev

### Easy Ensemble Classifier

In [196]:
# Train the Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [197]:
# Calculated the balanced accuracy score
predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9949433304272014


In [198]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,0,87
Actual 1,0,17118


In [200]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.00      0.00      0.00        87
    low_risk       0.99      1.00      1.00     17118

    accuracy                           0.99     17205
   macro avg       0.50      0.50      0.50     17205
weighted avg       0.99      0.99      0.99     17205



In [201]:
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00        87
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17118

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

   Easy Ensemble Classifier

2. Which model had the best recall score?

    Balanced Random Forest Classifier

3. Which model had the best geometric mean score?

    Both were 0

4. What are the top three features?

 total_rec_prncp,
 last_pymnt_amnt,
 total_rec_int